In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys
# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 


CommitFit


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



1776


In [3]:
train_change_feautre = train.iloc[:, 4:]
test_change_feautre = test.iloc[:, 4:]

train_code_change = train_change_feautre.values
test_code_change = test_change_feautre.values

In [4]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [5]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [8]:
len(train)

17

In [9]:
train['label'].value_counts()

label
Corrective    6
Perfective    6
Adaptive      5
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
Corrective    597
Perfective    594
Adaptive      585
Name: count, dtype: int64

In [11]:
len(train_dataset)

41

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}

In [13]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [14]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
252,a77138a065b1b594fdb7351f9503be4496995b97,intellij-community,fixed django template commenter again (PY-1949)--,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,7ed444678012a695278f54ab86ca4a5dd73537c7,intellij-community,one more test that fails; some more code moved...,Perfective,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1425,3f3ff95b46cb0ef97704396501b7c05b778cc448,Vala,cairo: Use closure param to ReadFunc and Write...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,454302cf65c3c617149f28aee819549dd02f3680,Mylyn Reviews,Move versions-task bridge to reviews\n,Perfective,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1474,74b5e8ad3b43949f122e6430fe576ebdd44cf04a,Vala,vapigen: add support for type_parameters attri...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85,44a27c5cd76f44e435671c69d1d8f60c42a2b420,hbase,HBASE-11920 Add CP hooks for ReplicationEndPoi...,Adaptive,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
545,daab3db062b9b8cddbb653fedca91288cf5a0684,kotlin,Add WITH_RUNTIME and WITH_REFLECT directives t...,Perfective,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1297,464fd741533548c6fec7393299185c293de0d862,Mylyn Reviews,392682: Show details for changesets\n\nUse the...,Adaptive,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
971,8c9507ec33019b88dd43ab47e1bd50d22332740b,Valadoc,girmetadata: allow to override <ONLINE>\n,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1157,a57bee46d7db4cd6257d45a0733cf575602b0bb3,tapiji,Creates `stable` branch.\n\nThe stable branch ...,Perfective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
model_id = "../../sentence-transformers/all-roberta-large-v1"

In [16]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [17]:
trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2024-05-26 17:12:30,570] A new study created in memory with name: no-name-ed4b325f-94f4-4363-b4b6-3d409982499a
Trial: {'learning_rate': 0.0002653120756985924}
model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1640
  Num epochs = 1
  Total optimization steps = 103
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/103 [00:00<?, ?it/s]

...

***** Running training *****
  Num examples = 1640
  Num epochs = 1
  Total optimization steps = 103
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/103 [00:00<?, ?it/s]

***** Running evaluation *****
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-05-26 17:18:43,095] Trial 9 finished with value: 0.42961711711711714 and parameters: {'learning_rate': 0.0005116335337247725}. Best is trial 3 with value: 0.5129504504504504.


+++++++++++ {'precision': 0.5771165685245533, 'recall': 0.42961711711711714, 'f1': 0.4792061766175218, 'accuracy': 0.42961711711711714}


In [21]:
best_run

BestRun(run_id='3', objective=0.5129504504504504, hyperparameters={'learning_rate': 1.0734484505621306e-06}, backend=<optuna.study.study.Study object at 0x7f74d7c461d0>)

In [18]:
best_run.hyperparameters

{'learning_rate': 1.0734484505621306e-06}

In [19]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1640
  Num epochs = 1
  Total optimization steps = 103
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/103 [00:00<?, ?it/s]

In [20]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

***** Running evaluation *****
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.5434445474572145,
 'recall': 0.5084459459459459,
 'f1': 0.5052964646374944,
 'accuracy': 0.5084459459459459}